In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_tuner.tuners import Hyperband
from keras_tuner.engine.hyperparameters import HyperParameters
from sklearn.model_selection import train_test_split

# declaring global constants
MAX_EPOCHS = 20
MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2
N_EPOCH_SEARCH = 20

num_classes = 10
input_shape = (32, 32, 3)
LOG_DIR = "./keras_tuner_logs"

In [20]:
xtrain_path = "./data/transformed_type2/xtrain.csv"
ytrain_path = "./data/transformed_type2/ytrain.csv"

X = pd.read_csv(xtrain_path, sep=',', header=None)
y = pd.read_csv(ytrain_path, sep=',', header=None)

X = X.to_numpy()
y = y.to_numpy()

# Assume that each row of `features` corresponds to the same row as `labels`.
assert X.shape[0] == y.shape[0]

n_features = X.shape[1]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42)

print(f"X_train shape ={X_train.shape}")
print(f"X_val shape ={X_val.shape}")
print(f"y_train shape ={y_train.shape}")
print(f"y_val shape ={y_val.shape}")

X_train shape =(303, 76)
X_val shape =(34, 76)
y_train shape =(303, 1)
y_val shape =(34, 1)


### Helper function to return a model

In [21]:
def build_model(hp):
    model = tf.keras.models.Sequential()
    
    #  configure keras tuner to choose a particular number of neurons
    hp_units = hp.Int('units', min_value=16, max_value=64, step=16)
    model.add(layers.Dense(units=hp_units, activation='relu', input_shape=(76,), kernel_initializer='glorot_uniform'))
    layers.Dropout(rate=0.5),  
    model.add(layers.Dense(1, activation="sigmoid"))
    
    
    # set the loss and optimizer
    model.compile(loss= tf.keras.losses.BinaryCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )),
                  metrics=["accuracy"])

    return model

### Start the search

In [22]:
# instantiate the search algo with 2 epochs per trial, objective and 20 model configurations.
# also set the directory for log files.

tuner = Hyperband(build_model, objective="val_accuracy", directory=LOG_DIR, max_epochs = MAX_EPOCHS,
                     executions_per_trial=EXECUTION_PER_TRIAL, project_name="hype_tune")

tuner.search(x=X_train, y=y_train, epochs=N_EPOCH_SEARCH, batch_size = 16, validation_data=(X_val, y_val))

INFO:tensorflow:Reloading Oracle from existing project ./keras_tuner_logs\hype_tune\oracle.json
INFO:tensorflow:Reloading Tuner from ./keras_tuner_logs\hype_tune\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [24]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
history = hypermodel.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 64 and the optimal learning rate for the optimizer
is 0.006997708001704712.

Epoch 1/20
10/10 [==============================] - 0s 14ms/step - loss: 0.4525 - accuracy: 0.7558 - val_loss: 0.1529 - val_accuracy: 1.0000
Epoch 2/20
10/10 [==============================] - 0s 3ms/step - loss: 0.2027 - accuracy: 0.9175 - val_loss: 0.0737 - val_accuracy: 1.0000
Epoch 3/20
10/10 [==============================] - 0s 3ms/step - loss: 0.1170 - accuracy: 0.9538 - val_loss: 0.0660 - val_accuracy: 1.0000
Epoch 4/20
10/10 [==============================] - 0s 3ms/step - loss: 0.0612 - accuracy: 0.9901 - val_loss: 0.0361 - val_accuracy: 1.0000
Epoch 5/20
10/10 [==============================] - 0s 3ms/step - loss: 0.0372 - accuracy: 0.9901 - val_loss: 0.0275 - val_accuracy: 1.0000
Epoch 6/20
10/10 [==============================] - 0s 3ms/step - loss: 0.0204 - accuracy: 1.0000 - val_loss: 0.01

In [28]:
print(f"Tuned Model accuracy= = {hypermodel.evaluate(X, y)[1]}")

11/11 [==============================] - 0s 1ms/step - loss: 9.5367e-04 - accuracy: 1.0000
Tuned Model accuracy= = 1.0


## predict on Unseen Test


In [31]:
Xtest = pd.read_csv("./data/transformed_type2/xtest.csv", sep=',', header=None)
ytest = pd.read_csv("./data/transformed_type2/ytest.csv", sep=',', header=None)

Xtest = Xtest.to_numpy()
ytest = ytest.to_numpy()

# Assume that each row of `features` corresponds to the same row as `labels`.
assert Xtest.shape[0] == ytest.shape[0]
print(f"Xtest shape ={Xtest.shape}")
print(f"ytest shape ={ytest.shape}")

print(f"Tuned Model accuracy on Hold-out Test Set = {hypermodel.evaluate(Xtest, ytest)[1]}")


Xtest shape =(38, 76)
ytest shape =(38, 1)
2/2 [==============================] - 0s 1ms/step - loss: 0.0014 - accuracy: 1.0000
Tuned Model accuracy on Hold-out Test Set = 1.0
